# Dataset discovery tools

This notebook shows some features to make the dataset discovery for CMS analysis easier. 
The rucio sytem is queried to look for dataset and access to the list of all available file replicas.

Users can exploit these tools at 2 different levels:
- low level: use the `rucio_utils` module directly to just query rucio
- high level: use the `DataDiscoveryCLI` class to simplify dataset query, replicas filters and uproot preprocessing with dask

In [2]:
!conda activate /depot/cms/users/yun79/conda_envs/test


usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'doctor', 'repoquery', 'env')


In [5]:
!conda env list

# conda environments:
#
                      *  /depot/cms/users/yun79/conda_envs/hmm_pocket_coffea
                         /depot/cms/users/yun79/conda_envs/test
base                     /opt/conda



In [1]:
import coffea
coffea.__version__

'2024.1.2'

# Using Rucio utils directly

In [1]:
from coffea.dataset_tools import rucio_utils
from coffea.dataset_tools.dataset_query import print_dataset_query
from rich.console import Console
from rich.table import Table

In [2]:
client = rucio_utils.get_rucio_client()
client

In [3]:
query = "/TTToSemiLeptonic_*_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9*/NANOAODSIM"

In [4]:
outlist, outtree = rucio_utils.query_dataset(
                query,
                client=client,
                tree=True,
                scope="cms",  
            )

outlist[1:5]

['/TTToSemiLeptonic_TuneCP5CR1_erdON_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM',
 '/TTToSemiLeptonic_TuneCP5CR2_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM',
 '/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM',
 '/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM']

In [5]:
# outtree["TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8"].keys()
# outtree["TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8"]['RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2']
outtree.keys()


dict_keys(['TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5CR1_erdON_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5CR2_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5_erdON_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5down_13TeV-powheg-pythia8', 'TTToSemiLeptonic_TuneCP5up_13TeV-powheg-pythia8', 'TTToSemiLeptonic_Vcb_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_hdampDOWN_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_hdampUP_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop166p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop169p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop171p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop173p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop175p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_mtop178p5_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_widthx0p55_TuneCP5_13TeV-powheg-pythia8', 'TTToSemiLeptonic_widthx0p7_TuneCP5_

Let's now pretty-print the results in a table using an utility function in the `dataset_query` module.

In [6]:
console = Console()
print_dataset_query(query, outtree, console)

              Query: /TTToSemiLeptonic_*_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9*/NANOAODSIM               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┓
┃ Name                              ┃ Tag                                                                        ┃┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇┩
│ TTToSemiLeptonic_TuneCP5CR1_13Te… │ (1) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NAN… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5CR1_erdO… │ (2) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NAN… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5CR2_13Te… │ (3) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NAN… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5_13TeV-p… │ (4) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NAN… ││
│                                   │ (5) RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v… ││
│                                   │ (6) RunIISummer20UL18NanoAODv9-PUForMUOVal_106X_upgrade2018_realistic_v16… ││
│                                   │ (7) RunIISummer20UL18NanoAODv9-PUForTRK_TRK_106X_upgrade2018_realistic_v1… ││
│                                   │ (8) RunIISummer20UL18NanoAODv9-PUForTRKv2_TRKv2_106X_upgrade2018_realisti… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5_erdON_1… │ (9) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NAN… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5down_13T… │ (10) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_TuneCP5up_13TeV… │ (11) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_Vcb_TuneCP5_13T… │ (12) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_hdampDOWN_TuneC… │ (13) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_hdampUP_TuneCP5… │ (14) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_mtop166p5_TuneC… │ (15) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_mtop169p5_TuneC… │ (16) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_mtop171p5_TuneC… │ (17) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NA… ││
├───────────────────────────────────┼────────────────────────────────────────────────────────────────────────────┼┤
│ TTToSemiLeptonic_mtop173p5_TuneC… │ (18) RunIISummer20

### Dataset replicas

Let's select one dataset and look for available replicas

In [7]:
dataset = outlist[0]
dataset

'/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM'

In [8]:
outlist[1]

'/TTToSemiLeptonic_TuneCP5CR1_erdON_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM'

Using the option `mode='full'` in the function `rucio_utils.get_dataset_file_replicas()` one gets all the available replicas. 

In [9]:
try:
    (
        outfiles,
        outsites,
        sites_counts,
    ) = rucio_utils.get_dataset_files_replicas(
        dataset,
        allowlist_sites=[],
        blocklist_sites=[],
        regex_sites=[],
        mode="first", 
        # mode="full",   # full or first. "full"==all the available replicas
        client=client,
    )
except Exception as e:
    print(f"\n[red bold] Exception: {e}[/]")

In [10]:
try:
    (
        outfiles_full,
        outsites_full,
        sites_counts_full,
    ) = rucio_utils.get_dataset_files_replicas(
        dataset,
        allowlist_sites=[],
        blocklist_sites=[],
        regex_sites=[],
        # mode="fist", 
        mode="full",   # full or first. "full"==all the available replicas
        client=client
    )
except Exception as e:
    print(f"\n[red bold] Exception: {e}[/]")

In [11]:
# set(outfiles).intersection(outfiles_full)
outfiles_full

[['root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0395E709-0B1D-5641-8D82-6B35014FA026.root',
  'root://dcache-cms-xrootd.desy.de:1094//store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0395E709-0B1D-5641-8D82-6B35014FA026.root',
  'root://cmsxrootd-kit-disk.gridka.de:1094///store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0395E709-0B1D-5641-8D82-6B35014FA026.root',
  'root://rdr.echo.stfc.ac.uk///store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0395E709-0B1D-5641-8D82-6B35014FA026.root',
  'root://cmsxrootd.hep.wisc.edu:1094///store/mc/RunIISummer20UL18NanoAODv9/TTToSemi

In [12]:
# type(outfiles[0])
outfiles

['root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0395E709-0B1D-5641-8D82-6B35014FA026.root',
 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0417474B-7A70-8D42-90D2-B95A04C14B0E.root',
 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/07EA3060-0086-3849-90E6-38223161ECE5.root',
 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/0D17B8E7-6BD9-DE47-B0E8-46349B56FD3C.root',
 'root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISum

In [13]:
# set(outsites).intersection(outsites_full)
outsites_full

[['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T1_UK_RAL_Disk',
  'T2_US_Wisconsin',
  'T1_RU_JINR_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T2_IT_Legnaro',
  'T1_UK_RAL_Disk',
  'T2_BR_UERJ',
  'T1_RU_JINR_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T2_IT_Legnaro',
  'T1_UK_RAL_Disk',
  'T2_BR_UERJ',
  'T1_RU_JINR_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T1_UK_RAL_Disk',
  'T2_US_Wisconsin',
  'T1_RU_JINR_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T2_IT_Legnaro',
  'T2_US_Vanderbilt',
  'T1_UK_RAL_Disk',
  'T1_RU_JINR_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DESY',
  'T1_DE_KIT_Disk',
  'T1_UK_RAL_Disk',
  'T2_US_Nebraska',
  'T1_RU_JINR_Disk',
  'T1_IT_CNAF_Disk',
  'T3_CH_PSI',
  'T3_KR_UOS'],
 ['T1_US_FNAL_Disk',
  'T2_DE_DE

In [14]:
def print_replicas(sites_counts):
    console.print(f"[cyan]Sites availability for dataset: [red]{dataset}")
    table = Table(title="Available replicas")
    table.add_column("Index", justify="center")
    table.add_column("Site", justify="left", style="cyan", no_wrap=True)
    table.add_column("Files", style="magenta", no_wrap=True)
    table.add_column("Availability", justify="center")
    table.row_styles = ["dim", "none"]
    Nfiles = len(outfiles)
    
    sorted_sites = dict(
        sorted(sites_counts.items(), key=lambda x: x[1], reverse=True)
    )
    for i, (site, stat) in enumerate(sorted_sites.items()):
        table.add_row(
            str(i), site, f"{stat} / {Nfiles}", f"{stat*100/Nfiles:.1f}%"
        )
    console.print(table)

In [15]:
print_replicas(sites_counts)

Sites availability for dataset: 
/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2
/NANOAODSIM

               Available replicas               
┏━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site      ┃ Files     ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T3_KR_UOS │ 294 / 294 │    100.0%    │
└───────┴───────────┴───────────┴──────────────┘

### Filtering sites
Grid sites can be filtered in 3 different ways
- **allowlist**:  if this list of specified, only the sites in the list are considered. No blocklist and regex are considered
- **blocklist**: if this list is specified, those sites are excluded from the replicas
- **regex_sites**: regex filter the sites to be considered, on top of the blocklist

In [16]:
# Example with allowlist
try:
    (
        outfiles,
        outsites,
        sites_counts,
    ) = rucio_utils.get_dataset_files_replicas(
        dataset,
        allowlist_sites=["T2_DE_DESY", "T1_US_FNAL_Disk"],
        blocklist_sites=[],
        regex_sites=None,
        mode="full",   # full or first. "full"==all the available replicas
        client=client,
    )
except Exception as e:
    print(f"\n[red bold] Exception: {e}[/]")

print_replicas(sites_counts)

Sites availability for dataset: 
/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2
/NANOAODSIM

                  Available replicas                  
┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site            ┃ Files     ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T2_DE_DESY      │ 294 / 294 │    100.0%    │
│   1   │ T1_US_FNAL_Disk │ 193 / 294 │    65.6%     │
└───────┴─────────────────┴───────────┴──────────────┘

In [17]:
# Example with blocklist
try:
    (
        outfiles,
        outsites,
        sites_counts,
    ) = rucio_utils.get_dataset_files_replicas(
        dataset,
        allowlist_sites=[],
        blocklist_sites=["T2_DE_DESY", "T3_CH_PSI"],
        regex_sites=None,
        mode="full",   # full or first. "full"==all the available replicas
        client=client,
    )
except Exception as e:
    print(f"\n[red bold] Exception: {e}[/]")

print_replicas(sites_counts)

Sites availability for dataset: 
/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2
/NANOAODSIM

                    Available replicas                    
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site                ┃ Files     ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T1_DE_KIT_Disk      │ 294 / 294 │    100.0%    │
│   1   │ T1_UK_RAL_Disk      │ 294 / 294 │    100.0%    │
│   2   │ T1_RU_JINR_Disk     │ 294 / 294 │    100.0%    │
│   3   │ T3_KR_UOS           │ 294 / 294 │    100.0%    │
│   4   │ T1_US_FNAL_Disk     │ 193 / 294 │    65.6%     │
│   5   │ T2_US_Nebraska      │ 99 / 294  │    33.7%     │
│   6   │ T1_IT_CNAF_Disk     │ 58 / 294  │    19.7%     │
│   7   │ T2_US_Purdue        │ 53 / 294  │    18.0%     │
│   8   │ T2_BE_IIHE          │ 50 / 294  │    17.0%     │
│   9   │ T2_US_MIT           │ 50 / 294  │    17.0%     │
│  10   │ T1_ES_PIC_Disk      │ 43 / 294  │    14.6%     │
│  11   │ T2_US_Vanderbilt    │ 40 / 294  │    13.6%     │
│  12   │ T2_BR_SPRACE        │ 39 / 294  │    13.3%     │
│  13   │ T2_US_Florida       │ 33 / 294  │    11.2%     │
│  14   │ T2_IT_Legnaro       │ 28 / 294  │     9.5%     │
│  15   │ T2_US_UCSD          │ 28 / 294  │     9.5%     │
│  16   │ T2_UA_KIPT          │ 26 / 294  │     8.8%     │
│  17   │ T2_US_Caltech       │ 24 / 294  │     8.2%     │
│  18   │ T2_US_Wisconsin     │ 22 / 294  │     7.5%     │
│  19   │ T2_TR_METU          │ 18 / 294  │     6.1%     │
│  20   │ T2_ES_CIEMAT        │ 17 / 294  │     5.8%     │
│  21   │ T2_DE_RWTH          │ 11 / 294  │     3.7%     │
│  22   │ T2_BR_UERJ          │ 7 / 294   │     2.4%     │
│  23   │ T2_UK_SGrid_Bristol │ 3 / 294   │     1.0%     │
│  24   │ T2_ES_IFCA          │ 2 / 294   │     0.7%     │
└───────┴─────────────────────┴───────────┴──────────────┘

In [18]:
# Example with regex
try:
    (
        outfiles,
        outsites,
        sites_counts,
    ) = rucio_utils.get_dataset_files_replicas(
        dataset,
        allowlist_sites=[],
        blocklist_sites=[],
        regex_sites= r"T[123]_(FR|IT|BE|CH|DE|ES|UK)_\w+",
        mode="full",   # full or first. "full"==all the available replicas
        client=client,
    )
except Exception as e:
    print(f"\n[red bold] Exception: {e}[/]")

print_replicas(sites_counts)

Sites availability for dataset: 
/TTToSemiLeptonic_TuneCP5CR1_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2
/NANOAODSIM

                    Available replicas                    
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site                ┃ Files     ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T2_DE_DESY          │ 294 / 294 │    100.0%    │
│   1   │ T1_DE_KIT_Disk      │ 294 / 294 │    100.0%    │
│   2   │ T1_UK_RAL_Disk      │ 294 / 294 │    100.0%    │
│   3   │ T3_CH_PSI           │ 294 / 294 │    100.0%    │
│   4   │ T1_IT_CNAF_Disk     │ 58 / 294  │    19.7%     │
│   5   │ T2_BE_IIHE          │ 50 / 294  │    17.0%     │
│   6   │ T1_ES_PIC_Disk      │ 43 / 294  │    14.6%     │
│   7   │ T2_IT_Legnaro       │ 28 / 294  │     9.5%     │
│   8   │ T2_ES_CIEMAT        │ 17 / 294  │     5.8%     │
│   9   │ T2_DE_RWTH          │ 11 / 294  │     3.7%     │
│  10   │ T2_UK_SGrid_Bristol │ 3 / 294   │     1.0%     │
│  11   │ T2_ES_IFCA          │ 2 / 294   │     0.7%     │
└───────┴─────────────────────┴───────────┴──────────────┘

# Using the DataDiscoveryCLI
Manipulating the dataset query and replicas is simplified by the `DataDiscoveryCLI` class in `dataset_query` module.

In [22]:
from coffea.dataset_tools import rucio_utils
from coffea.dataset_tools.dataset_query import print_dataset_query
from rich.console import Console
from rich.table import Table
from coffea.dataset_tools.dataset_query import DataDiscoveryCLI

In [23]:
dataset_definition = {
    "/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X*/NANOAODSIM": {"short_name": "ZJets",
                                                                                                   "metadata": {"xsec": 100.0,"isMC":True}},
    "/SingleMuon/Run2018C-UL20*_MiniAODv2_NanoAODv9_GT36*/NANOAOD": {"short_name": "SingleMuon", "metadata": {"isMC":False}}
}


The dataset definition is passed to a `DataDiscoveryCLI` to automatically query rucio and get replicas

In [24]:
ddc = DataDiscoveryCLI()
ddc.load_dataset_definition(dataset_definition, 
                           query_results_strategy="all",
                           replicas_strategy="round-robin")

Processing query: /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X*/NANOAODSIM

      Query: /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X*/NANOAODSIM       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┓
┃ Name                                   ┃ Tag                                                                   ┃┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇┩
│ DYJetsToLL_M-50_TuneCP5_13TeV-amcatnl… │ (1) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v… ││
└────────────────────────────────────────┴───────────────────────────────────────────────────────────────────────┴┘

Use the command select to selected the datasets

Selected datasets:

- (1) 
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-
v2/NANOAODSIM

...

                   Available replicas                   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site                ┃ Files   ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T2_DE_DESY          │ 67 / 67 │    100.0%    │
│   1   │ T3_KR_KISTI         │ 67 / 67 │    100.0%    │
│   2   │ T2_TW_NCHC          │ 67 / 67 │    100.0%    │
│   3   │ T2_BE_IIHE          │ 67 / 67 │    100.0%    │
│   4   │ T2_US_Purdue        │ 67 / 67 │    100.0%    │
│   5   │ T2_ES_CIEMAT        │ 67 / 67 │    100.0%    │
│   6   │ T3_FR_IPNL          │ 67 / 67 │    100.0%    │
│   7   │ T1_US_FNAL_Disk     │ 61 / 67 │    91.0%     │
│   8   │ T2_UK_London_IC     │ 39 / 67 │    58.2%     │
│   9   │ T1_FR_CCIN2P3_Disk  │ 38 / 67 │    56.7%     │
│  10   │ T2_US_Caltech       │ 26 / 67 │    38.8%     │
│  11   │ T2_CH_CERN          │ 25 / 67 │    37.3%     │
│  12   │ T2_DE_RWTH          │ 22 / 67 │    32.8%     │
│  13   │ T1_IT_CNAF_Disk     │ 20 / 67 │    29.9%     │
│  14   │ T2_US_Wisconsin     │ 16 / 67 │    23.9%     │
│  15   │ T2_US_Florida       │ 16 / 67 │    23.9%     │
│  16   │ T2_US_Nebraska      │ 13 / 67 │    19.4%     │
│  17   │ T2_TR_METU          │ 11 / 67 │    16.4%     │
│  18   │ T1_DE_KIT_Disk      │ 11 / 67 │    16.4%     │
│  19   │ T2_UK_SGrid_RALPP   │ 6 / 67  │     9.0%     │
│  20   │ T2_IT_Legnaro       │ 6 / 67  │     9.0%     │
│  21   │ T2_ES_IFCA          │ 4 / 67  │     6.0%     │
│  22   │ T2_FR_IPHC          │ 2 / 67  │     3.0%     │
│  23   │ T2_UK_London_Brunel │ 1 / 67  │     1.5%     │
└───────┴─────────────────────┴─────────┴──────────────┘

Replicas for /SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9_GT36-v1/NANOAOD
├── T2_TW_NCHC
│   ├── root://se01.grid.nchc.org.tw//cms//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v
│   │   1/2520000/0144EC47-BFA3-EA43-BF05-BD4248ED6031.root
│   ├── root://se01.grid.nchc.org.tw//cms//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v
│   │   1/2520000/26FC8C40-EA29-804C-B17D-84FB1C6BC505.root
│   ├── root://se01.grid.nchc.org.tw//cms//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v
│   │   1/2520000/2DA9130E-8423-304C-9902-1E42CD72E658.root
│   ├── root://se01.grid.nchc.org.tw//cms//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v
│   │   1/2520000/8C8690F8-4FEE-1047-85F4-29E414B3D12C.root
│   └── root://se01.grid.nchc.org.tw//cms//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v
│       1/2520000/D8D41BBC-D514-D342-A514-CCF48575D184.root
├── T1_US_FNAL_Disk
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/0C9615C1-7EE6-CD44-8FC0-04F63B2C16FD.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/3FE5B677-9AB3-0245-A1CF-4B320592F18F.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/410C32AB-DEB5-404F-BC6B-92E8F560563F.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/B1B449CE-5952-8347-A9A7-35FE231D0C72.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/ECD4877E-707B-EA43-A38B-D1B700FBDE79.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/ED95384D-9D3D-AE45-8425-C4C080E691C5.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│   │   Dv9_GT36-v1/2520000/F16A9138-7563-E540-B6AD-8A8A688B3830.root
│   └── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAO
│       Dv9_GT36-v1/2520000/FE5EEFA5-C07A-5C44-B66D-5B31BE02C7D3.root
├── T2_ES_CIEMAT
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/12FAE9F1-7139-924C-A8DE-9699A00FC994.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/2747DEFE-A247-1F42-B0EF-E7B7F1D3FCD6.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/42DC0F42-82E8-BE47-B04D-544B67274829.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/51515E3C-C640-3A4C-A16C-DC267FD142BF.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/8369B0EA-E4CC-AC4D-BD3F-0679B3310E09.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/BCBF89A2-329C-744B-A38F-139EA8F94007.root
│   ├── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│   │   /2520000/D7875684-9F26-084E-9B2B-5E9BB5D353E8.root
│   └── root://gaexrdoor.ciemat.es:1094///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1
│       /2520000/F1B3977A-E777-EC4D-8FC7-981FE4ED5E0C.root
├── T2_IT_Legnaro
│   ├── root://t2-xrdcms.lnl.infn.it:7070///store/data/Run2018C/SingleMuon/NANOAOD/UL

Selected datasets:

                                                 Selected datasets                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┳┓
┃ … ┃ Dataset                                                                                                   ┃┃┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇╇┩
│ 1 │ /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realisti… │││
│ 2 │ /SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9_GT36-v1/NANOAOD                                           │││
└───┴───────────────────────────────────────────────────────────────────────────────────────────────────────────┴┴┘

{'/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM': {'files': {'root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/13D0AD97-6B32-CB4C-BA87-5E37BA4CF20E.root': 'Events',
   'root://cmsxrd.ts.infn.it:1094///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00C9792D-ACD2-2547-BB04-097F0C4E47E3.root': 'Events',
   'root://deepthought.crc.nd.edu///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00EA9563-5449-D24E-9566-98AE8E2A61AE.root': 'Events',
   'root://cms-xrdr.sdfarm.kr:1094//xrd//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAO

### Filtering sites

Sites filtering works in a very similar way for `DataDiscoveryCLI`

In [25]:
ddc = DataDiscoveryCLI()
ddc.do_regex_sites(r"T[123]_(CH|IT|UK|FR|DE)_\w+")
ddc.load_dataset_definition(dataset_definition, 
                           query_results_strategy="all",
                           replicas_strategy="round-robin")

New sites regex: T[123]_(CH|IT|UK|FR|DE)_\w+

Processing query: /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X*/NANOAODSIM

      Query: /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X*/NANOAODSIM       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┓
┃ Name                                   ┃ Tag                                                                   ┃┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇┩
│ DYJetsToLL_M-50_TuneCP5_13TeV-amcatnl… │ (1) RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v… ││
└────────────────────────────────────────┴───────────────────────────────────────────────────────────────────────┴┘

Use the command select to selected the datasets

Selected datasets:

...

                   Available replicas                   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site                ┃ Files   ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T2_DE_DESY          │ 67 / 67 │    100.0%    │
│   1   │ T3_FR_IPNL          │ 67 / 67 │    100.0%    │
│   2   │ T2_UK_London_IC     │ 39 / 67 │    58.2%     │
│   3   │ T1_FR_CCIN2P3_Disk  │ 38 / 67 │    56.7%     │
│   4   │ T2_CH_CERN          │ 25 / 67 │    37.3%     │
│   5   │ T2_DE_RWTH          │ 22 / 67 │    32.8%     │
│   6   │ T1_IT_CNAF_Disk     │ 20 / 67 │    29.9%     │
│   7   │ T1_DE_KIT_Disk      │ 11 / 67 │    16.4%     │
│   8   │ T2_UK_SGrid_RALPP   │ 6 / 67  │     9.0%     │
│   9   │ T2_IT_Legnaro       │ 6 / 67  │     9.0%     │
│  10   │ T2_FR_IPHC          │ 2 / 67  │     3.0%     │
│  11   │ T2_UK_London_Brunel │ 1 / 67  │     1.5%     │
└───────┴─────────────────────┴─────────┴──────────────┘

Replicas for /SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9_GT36-v1/NANOAOD
├── T1_IT_CNAF_Disk
│   ├── root://xrootd-cms.infn.it:1194///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1/
│   │   2520000/0144EC47-BFA3-EA43-BF05-BD4248ED6031.root
│   ├── root://xrootd-cms.infn.it:1194///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1/
│   │   2520000/3FE5B677-9AB3-0245-A1CF-4B320592F18F.root
│   ├── root://xrootd-cms.infn.it:1194///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1/
│   │   2520000/648ECD9C-8AAA-BB46-8683-C8987CCC73B9.root
│   ├── root://xrootd-cms.infn.it:1194///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1/
│   │   2520000/74A75B73-E5B8-C942-BBC9-1DDDD7F752FB.root
│   └── root://xrootd-cms.infn.it:1194///store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAODv2_NanoAODv9_GT36-v1/
│       2520000/B3487FE0-B172-AD47-A13A-388C0A9BF93F.root
├── T1_FR_CCIN2P3_Disk
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/0C9615C1-7EE6-CD44-8FC0-04F63B2C16FD.root
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/152C304A-97AD-1649-BCB6-3EA0CCD0DD33.root
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/2747DEFE-A247-1F42-B0EF-E7B7F1D3FCD6.root
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/62789325-3C0B-FC4D-B578-B41A396399E4.root
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/78AC6A39-C303-EB44-9264-71819CC70FCC.root
│   ├── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│   │   18_MiniAODv2_NanoAODv9_GT36-v1/2520000/8C8690F8-4FEE-1047-85F4-29E414B3D12C.root
│   └── root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/data/Run2018C/SingleMuon/NANOAOD/UL20
│       18_MiniAODv2_NanoAODv9_GT36-v1/2520000/F6E44EA5-F4C6-E746-AD43-7A263F1E316E.root
├── T3_FR_IPNL
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/12FAE9F1-7139-924C-A8DE-9699A00FC994.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/1CEB718A-7DC1-C74A-A7BE-A3C8D9FA785A.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/1DD0FAC6-3087-E44E-ABCB-8AF812C1310D.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/42DC0F42-82E8-BE47-B04D-544B67274829.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/51515E3C-C640-3A4C-A16C-DC267FD142BF.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/7DEA3718-B7BC-EE42-A8BE-11C62BB8536D.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/81CEA7BA-9E66-BC4F-A96F-32642D59B653.root
│   ├── root://lyogrid06.in2p3.fr//dpm/in2p3.fr/home/cms/data//store/data/Run2018C/SingleMuon/NANOAOD/UL2018_MiniAO
│   │   Dv2_NanoAODv9_GT36-v1/2520000/8223C4A3-D4BD-6A4B-A513-54B6668C71

Selected datasets:

                                                 Selected datasets                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┳┓
┃ … ┃ Dataset                                                                                                   ┃┃┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇╇┩
│ 1 │ /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realisti… │││
│ 2 │ /SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9_GT36-v1/NANOAOD                                           │││
└───┴───────────────────────────────────────────────────────────────────────────────────────────────────────────┴┴┘

{'/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM': {'files': {'root://grid-cms-xrootd.physik.rwth-aachen.de:1094///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/13D0AD97-6B32-CB4C-BA87-5E37BA4CF20E.root': 'Events',
   'root://xrootd-cms.infn.it:1194///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00C9792D-ACD2-2547-BB04-097F0C4E47E3.root': 'Events',
   'root://dcache-cms-xrootd.desy.de:1094//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00EA9563-5449-D24E-9566-98AE8E2A61AE.root': 'Events',
   'root://t2-xrdcms.lnl.infn.it:7070///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NA

In [27]:
ddc.do_list_selected()

Selected datasets:

                                                 Selected datasets                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┳┓
┃ … ┃ Dataset                                                                                                   ┃┃┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇╇┩
│ 1 │ /DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realisti… │││
│ 2 │ /SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9_GT36-v1/NANOAOD                                           │││
└───┴───────────────────────────────────────────────────────────────────────────────────────────────────────────┴┴┘

### Save the replicas metadata

In [28]:
ddc.do_save("replicas_info.json")

File replicas_info.json saved!

## DataDiscoveryCLI from shell

The DataDiscoveryCLI can be used directly from CLI

In [35]:
!python -m coffea.dataset_tools.dataset_query --help

usage: dataset_query.py [-h] [--cli] [-d DATASET_DEFINITION] [-o OUTPUT]
                        [-fo FILESET_OUTPUT] [-p] [--step-size STEP_SIZE]
                        [--dask-cluster DASK_CLUSTER]
                        [-as ALLOW_SITES [ALLOW_SITES ...]]
                        [-bs BLOCK_SITES [BLOCK_SITES ...]] [-rs REGEX_SITES]
                        [--query-results-strategy QUERY_RESULTS_STRATEGY]
                        [--replicas-strategy REPLICAS_STRATEGY]

options:
  -h, --help            show this help message and exit
  --cli                 Start the dataset discovery CLI
  -d DATASET_DEFINITION, --dataset-definition DATASET_DEFINITION
                        Dataset definition file
  -o OUTPUT, --output OUTPUT
                        Output name for dataset discovery output (no fileset
                        preprocessing)
  -fo FILESET_OUTPUT, --fileset-output FILESET_OUTPUT
                        Output name for fileset
  -p, --preprocess      Preprocess with d

In [33]:
!python -m coffea.dataset_tools.dataset_query --cli  -d dataset_definition.json
# !python -m coffea.dataset_tools.dataset_query --cli --help

Traceback (most recent call last):
  File "/depot/cms/kernels/coffea2024/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/depot/cms/kernels/coffea2024/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/depot/cms/kernels/coffea2024/lib/python3.10/site-packages/coffea/dataset_tools/dataset_query.py", line 699, in <module>
    with open(args.dataset_definition) as file:
FileNotFoundError: [Errno 2] No such file or directory: 'dataset_definition.json'


## Preprocess the fileset with dask

The replicas metadata contain the file location in the CMS grid. 
This info can be **preprocessed** with uproot and dask-awkward to extract the **fileset**. Practically a fileset is a collection of metadata about the file location, file name, chunks splitting, that can be used directly to configure the uproot reading. 

This step replaces the preprocessing step in coffea 0.7.x. The output of the preprocessing can be used directly to start an analysis with dask-awkward.

The preprocessing is performed locally with multiple processes if `dask_cluster==None`, but a pre-existing dask cluster url can be passed.

In [53]:
fileset_total = ddc.do_preprocess(output_file="fileset",
                  step_size=10000,  #chunk size for files splitting
                                  dask_cluster=None,
                  align_to_clusters=False)

TypeError: DataDiscoveryCLI.do_preprocess() got an unexpected keyword argument 'dask_cluster'

In [36]:
import gzip
import json
with gzip.open("fileset_available.json.gz", "rt") as file:
    fileset_available = json.load(file)

In [51]:
# len(fileset_available)
# fileset_available[dataset].keys()
fileset_available[dataset]['files'].keys()

dict_keys(['root://grid-cms-xrootd.physik.rwth-aachen.de:1094///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/13D0AD97-6B32-CB4C-BA87-5E37BA4CF20E.root', 'root://xrootd-cms.infn.it:1194///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00C9792D-ACD2-2547-BB04-097F0C4E47E3.root', 'root://dcache-cms-xrootd.desy.de:1094//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00EA9563-5449-D24E-9566-98AE8E2A61AE.root', 'root://t2-xrdcms.lnl.infn.it:7070///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/04724383-31E5-0544-8505-F60FF4A8673E.root', 'root://dcache-cms-xrootd.desy.de:1094//store/mc/RunIISummer20UL18NanoAOD

In [45]:
dataset 'form'

'/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM'

In [41]:
dataset = '/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM'
for i, (file, meta) in enumerate(fileset_available[dataset]["files"].items()):
    print(file, meta)    
    if i>0: break

root://grid-cms-xrootd.physik.rwth-aachen.de:1094///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/13D0AD97-6B32-CB4C-BA87-5E37BA4CF20E.root {'object_path': 'Events', 'steps': [[0, 10000], [10000, 20000], [20000, 30000], [30000, 40000], [40000, 50000], [50000, 59081]], 'uuid': 'fbe50b00-1f7e-11ec-97b8-2bbee183beef'}
root://xrootd-cms.infn.it:1194///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/00C9792D-ACD2-2547-BB04-097F0C4E47E3.root {'object_path': 'Events', 'steps': [[0, 10000], [10000, 20000], [20000, 30000], [30000, 40000], [40000, 50000], [50000, 60000], [60000, 70000], [70000, 80000], [80000, 90000], [90000, 100000], [100000, 110000], [110000, 120000], [120000, 130000], [130000, 138192]], 'uuid': '938a4fe2-1d77-11ec-bddf-59319e86beef'}


In [42]:
len(fileset_available[dataset]["files"].items())

204